# Snow Pit Data Access and SWE Calculation

This notebook is designed to access data from snow pits gathered during the SnowEx field campaigns in Alaska. A combination of snow depth and density is gathered to estimate SWE across the Toolik field site (UKT).

In this example, the snow pit data is accessed using the SnowEx Database and `snowexsql`. The code is adapted from a SnowEx Database example found here: https://snowexsql.readthedocs.io/en/latest/gallery/plot_pit_swe_example.html

In [ ]:
import earthaccess
import pandas as pd
import geopandas as gpd
import os
import tempfile
from shapely.geometry import Point
import shutil

In [ ]:
# Authenticate with Earthdata Login servers
auth = earthaccess.login(strategy="interactive")

# Search for granules
results = earthaccess.search_data(
    short_name="SNEX20_SD",
    temporal=('2020-01-01', '2020-02-01'),
)

In [ ]:
display(results[0])

In [ ]:
# Create a temporary directory for downloads
temp_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {temp_dir}")

# Download the data to the temp directory
downloaded_files = earthaccess.download(
    results,
    local_path="/home/jovyan/tmp",
)
print(f"Downloaded {len(downloaded_files)} files to {temp_dir}")

# Process CSV files and convert to GeoDataFrame
gdf = []
csv_files = [file for file in downloaded_files if file.endswith('.csv')]
if csv_files:
    for i, csv_file in enumerate(csv_files):
        print(f"Processing: {os.path.basename(csv_file)}")

        # Read the csv file
        tmp_df = pd.read_csv(csv_file)

        # Convert to GeoDataFrame
        geometry = [Point(xy) for xy in zip(df['Easting'], df['Northing'])]
        tmp_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:32612")

        # Add to final GeoDataFrame
        gdf = pd.concat([gdf, tmp_gdf])

shutil.rmtree(temp_dir)

In [ ]:
from snowexsql.api import PointMeasurements, LayerMeasurements

# Instantiate the class to use the properties!
measurements = PointMeasurements()

# Get the unique data names/types in the table
results = measurements.all_types
print('Available types = {}'.format(', '.join([str(r) for r in results])))

# Get the unique instrument in the table
results = measurements.all_instruments
print('\nAvailable Instruments = {}'.format(', '.join([str(r) for r in results])))

# Get the unique dates in the table
results = measurements.all_dates
print('\nAvailable Dates = {}'.format(', '.join(sorted([str(r) for r in results]))))

# Get the unique site names in the table
results = measurements.all_site_names
print('\nAvailable sites = {}'.format(', '.join([str(r) for r in results])))

In [ ]:
# Pick the first one we find
#site_id = LayerMeasurements().all_site_ids
site_id = LayerMeasurements()

# Query the database, we only need one point to get a site id and its geometry
#site_df = LayerMeasurements.from_filter(site_id=site_id, limit=1)

# Print it out 
site_id.all_site_ids

In [ ]:
data_type = 'depth'

In [ ]:
import pandas as pd
site_df = pd.DataFrame()
for site in LayerMeasurements().all_site_ids:
    tmp = PointMeasurements.from_filter(site_id=site, type='depth', limit=1)
    try:
        tmp = tmp.to_crs("EPSG:4326")
    except:
        tmp = tmp.set_crs("EPSG:4326")

    site_df = pd.concat([site_df, tmp])

# Pick the first one we find
#site_id = LayerMeasurements().all_site_ids

# Query the database, we only need one point to get a site id and its geometry
#site_df = LayerMeasurements.from_filter(site_id=site_id)

# Print it out 
#site_df

In [ ]:
site_df.explore()

In [ ]:
site_df['site_id'][site_df['id']==7643177].values

In [ ]:
# Import in our two classes to access the db
from snowexsql.api import LayerMeasurements
from datetime import datetime 

# Find some density pit measurements at the Boise site in december 2019.
df = PointMeasurements.from_filter(
    type="depth",
    site_id="Skyway Tree",
    #date_greater_equal=datetime(2018, 1, 1),
    #date_less_equal=datetime(2022, 12, 1),
    #instrument="magnaprobe",
    limit=1000
)

In [ ]:
df

In [ ]:
df.plot(column='value', cmap='Blues')

In [ ]:
df_area = PointMeasurements.from_area(pt=df.geometry[0], type=data_type, limit=1000, buffer=200)

In [ ]:
df_area.keys()

In [ ]:
df_area.plot(column='value')

In [ ]:
df = df[df.value != 'None']
df['value'] = df['value'].astype(float)
print(df[['site_id', 'value']].groupby(by='site_id').mean())

In [ ]:
point = Point(df.iloc[0].easting, df.iloc[0].northing)

In [ ]:
df = PointMeasurements.from_filter(type='two_way_travel', limit=100)
df

In [ ]:
df.plot()